# Эксперимент №1: расчет характеристик тандемной сети с узлами MAP/PH/1/N с помощью имитационного моделирования и машинного обучения

В этом эксперименте мы рассчитаем различные характеристики сетей с линейной топологией, на вход которых поступает MAP-поток, а обслуживание имеет распределение фазового типа. Сначала мы рассчитаем характеристики на заданной сетке статистических параметров с помощью имитационного моделирования сети, а затем используем полученные результаты для обучения нейросетевых и других моделей ML, которые сможем использовать для очень быстрой оценки характеристик сетей. Например, такой подход полезен при нахождении решений задач оптимизации топологии, когда характеристики сетей с линейной топологией являются ограничениями в алгоритме ветвей и границ. 

## Подготовка ноутбука

Импортируем все необходимые библиотеки, настроим флаги и константы, которыми будем пользоваться в эксперименте, и определим служебные функции, чтобы не отвлекаться на них в расчетной части эксперимента.

In [1]:
# Если что-то меняется в коде, мы хотим это сразу видеть здесь
%load_ext autoreload
%autoreload 2

In [2]:
import os

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

from tabulate import tabulate
from tqdm.notebook import tqdm

from itertools import product
from collections import namedtuple
import time
import numpy as np
import pandas as pd

# Подключаем локальные пакеты
from pyqumo.random import Exponential, HyperExponential, Erlang
from pyqumo.cqumo.sim import simulate_tandem

In [3]:
# Настраиваем matplotlib
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})

Определим переменные окружения, которые будут использоваться в эксперименте.

In [4]:
# Нужно ли пересчитывать все, или можно использовать результаты из файлов
FORCE_SIMULATION = True
SIM_FILE_NAME = '01_tandem_simulation.csv'
SIM_FILE_DIR = 'data'
SIM_FILE_PATH = os.path.join(SIM_FILE_DIR, SIM_FILE_NAME)

# Цветовая схема для графиков
CMAP_NAME = 'viridis'

In [5]:
def get_color(x):
    """
    Получить цвет из текущей карты.
    """
    return cm.get_cmap(CMAP_NAME)(x)

## Вызов имитационной модели, чтение и сохранение результатов

Определим полезные функции, которые нам потребуются для работы с данными имитационного моделирования:

- `load_sim_data()`: читает файл `SIM_FILE_NAME`, если он существует, или создает новый `DataFrame` для хранения данных о результатах имитационного моделирования.
- `save_sim_data()`: сохраняет результаты, записанные в `DataFrame`, в файл `SIM_FILE_PATH`.

In [6]:
def save_sim_data(df: pd.DataFrame):
    """
    Сохранить в файл данные о результатах имитационного моделирования.
    """
    if not os.path.exists(SIM_FILE_DIR):
        os.makedirs(SIM_FILE_DIR)
    df.to_csv(SIM_FILE_PATH, index_label='Id')

    
def load_sim_data() -> pd.DataFrame:
    """
    Загрузить данные о резулдьтатах имитационного моделирования.
    """       
    if os.path.exists(SIM_FILE_PATH):
        return pd.read_csv(SIM_FILE_PATH)
    return pd.DataFrame(columns=('ArrAvg', 'ArrStd', 'ArrCv', 'SrvAvg', 'SrvStd', 'SrvCv', 'Rho', 
                                 'NetSize', 'Capacity', 'NumPackets',
                                 'AvgDelay', 'DeliveryProb'))

maps_df = load_sim_data()
maps_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ArrAvg        0 non-null      object
 1   ArrStd        0 non-null      object
 2   ArrCv         0 non-null      object
 3   SrvAvg        0 non-null      object
 4   SrvStd        0 non-null      object
 5   SrvCv         0 non-null      object
 6   Rho           0 non-null      object
 7   NetSize       0 non-null      object
 8   Capacity      0 non-null      object
 9   NumPackets    0 non-null      object
 10  AvgDelay      0 non-null      object
 11  DeliveryProb  0 non-null      object
dtypes: object(12)
memory usage: 0.0+ bytes


Функция `get_or_simulate(df, *, force=False, **kwargs)` работает так: если данные об эксперименте с заданными параметрами уже есть в `DataFrame`, не задан `force=True`, не установлена константа `FORCE_SIMULATION=True` и не указано значение `num_packets`, большее, чем в существующей записи, вернуть это значение. Если же хотя бы одно из этих условий нарушено, выполнить моделирование.

In [ ]:
def get_or_simulate(df: pd.DataFrame, *, force=False, num_packets=None, **kwargs) -> pd.DataFrame:
    """
    Получить строку с данными (параметрами и характеристиками) сети с заданными параметрами.
    
    Возможные ключи в kwargs:
    
    - arr_avg
    - arr_std
    - arr_cv
    - srv_avg
    - srv_std
    - srv_cv
    - rho
    - net_size
    - capacity
    - avg_delay
    - delivery_prob
    
    Если константа `FORCE_SIMULATION=True` или аргумент `force=True`, или параметр `num_packets` больше, 
    чем значение в `df` для указанных аргументов из `kwargs`, то будет выполнена имитационная модель,
    результат записан в `df` и возвращен.
    
    Если указано слишком мало параметров
    """

## Определяем сетку параметров модели

Будем считать, что сеть состоит из $L$ узлов, пакеты поступают на первую станцию и передаються по сети до тех пор, пока не будут обслужены последней станцией, либо не будут потерены из-за переполнения буфера на очередном узле.

Сеть будем описывать с помощью шести параметров:

- среднее значение интервалов между последовательными поступлениями пакетов в сеть ($\lambda^{-1}$)
- стандартное отклонение величин интервалов между последовательными поступлениями пакетов в сеть ($\sigma_A$)
- средняя длительность обслуживания ($\mu^{-1}$)
- стандартное отклонение длительности обслуживания ($\sigma_S$)
- число станций в сети ($L$)
- емкость очередей ($N$)

Из этих параметров можно полуить производные значения, которые оказываются более удобными при анализе:

- загрузка первой станции $\rho = \lambda / \mu$
- коэффициент вариации интервалов между поступлениями пакетов $c_A = \lambda \sigma_A$
- коэффициент вариации времени обслуживания $c_S = \mu \sigma_S$

Сетку будем задавать на множестве параметров $(\lambda^{-1}, \sigma_A, \mu^{-1}, \sigma_S, L, N)$.

Чтобы не пересчитывать результаты каждый раз заново, будем сохранять результаты расчета в файл `data/01_tandem_simulations.csv`. Если такого файла нет, или установлен флаг `FORCE_SIMULATION = True`, то каждая точка сетки будет рассчитана заново, а результаты расчета будут сохранены в файл. В противном случае расчет будет выполняться только в тогда, когда точки нет в файле, или в текущем расчете предполагается моделировать больше пакетов, то есть получить более точные результаты.

In [35]:
Params = namedtuple('Params', [
    'arrival_avg',
    'arrival_std',
    'service_avg',
    'service_std',
    'num_stations',
    'queue_capacity'
])

In [37]:
ARRIVAL_AVG = np.asarray([10, 20])
ARRIVAL_STD = np.asarray([1, 5, 10])
SERVICE_AVG = np.asarray([2.5, 5])
SERVICE_STD = np.asarray([1, 2.5, 5, 7.5, 10])
NUM_STATIONS = np.asarray([5])
QUEUE_CAPACITY = np.asarray([10])

# Build the parameters grid:
parameters = [
    Params(arrival_avg, arrival_std, service_avg, service_std, num_stations, queue_capacity)
    for (arrival_avg, arrival_std, service_avg, service_std, num_stations, queue_capacity) 
    in product(ARRIVAL_AVG, ARRIVAL_STD, SERVICE_AVG, SERVICE_STD, NUM_STATIONS, QUEUE_CAPACITY)
]
print(f"Defined {len(parameters)} parameters grid points")

Defined 60 parameters grid points


In [38]:
# Run the simulation
results = []  # store (params, ret), where `ret` is an instance of `pyqumo.sim.tandem.Result`
NUM_PACKETS = 100000

# This function returns the most appropriate distribution:
def get_distribution(avg, std):
    cv = std / avg
    if cv == 1:
        return Exponential(avg)
    if cv > 1:
        return HyperExponential.fit(avg, std)
    return Erlang.fit(avg, std)


for params in parameters:
    arrival = get_distribution(params.arrival_avg, params.arrival_std)
    services = [
        get_distribution(params.service_avg, params.service_std)
        for _ in range(params.num_stations)
    ]
    ret = simulate_tandem(arrival, services, params.queue_capacity, NUM_PACKETS)
    results.append((params, ret))


In [39]:
# Build a table:
from tabulate import tabulate

rows = []
for (param, ret) in results:
    rows.append((param.arrival_avg, param.arrival_std, param.service_avg, param.service_std, param.queue_capacity, 
                 param.num_stations, ret.delivery_delays[0].avg, ret.delivery_delays[0].std, ret.delivery_prob[0]))
print(tabulate(rows, headers=(
    'Arr.avg.', 'Arr.std.',
    'Srv.avg.', 'Srv. std.',
    'Queue capacity', 'Num. stations',
    'Delay avg.', 'Delay std.',
    'Delivery P.'
)))

  Arr.avg.    Arr.std.    Srv.avg.    Srv. std.    Queue capacity    Num. stations    Delay avg.    Delay std.    Delivery P.
----------  ----------  ----------  -----------  ----------------  ---------------  ------------  ------------  -------------
        10           1         2.5          1                  10                5     12.0198        2.18141       1
        10           1         2.5          2.5                10                5      1.99511       0.890679      1
        10           1         2.5          5                  10                5     20.3976       15.2676        0.99997
        10           1         2.5          7.5                10                5     33.5042       31.4547        0.987119
        10           1         2.5         10                  10                5     46.1059       48.7794        0.938449
        10           1         5            1                  10                5     25.1002        2.18963       1
        10          